In [ ]:
!pip install --upgrade ultralytics==8.0.186
!pip install torch torchvision torchaudio
!pip install opencv-python-headless
!pip install albumentations==1.4

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from ultralytics import YOLO
import numpy as np
from google.colab import drive
import os
import albumentations
import cv2
import torch.nn.functional as F
import matplotlib.pyplot as plt

drive.mount('/content/drive')

In [ ]:
dataset_yolo = '/content/drive/My Drive/2024 Fall/COMS 6730/Projects/Senior-Design-VIAD'
epoch_number = 50
weights_path = '/content/drive/My Drive/2024 Fall/COMS 6730/Projects/YoloV8/yolov8n.pt'
data_yaml_path = '/content/drive/My Drive/2024 Fall/COMS 6730/Projects/Senior-Design-VIAD/data.yaml'
saved_path = f'trained_model'
results_folder_name = f"{dataset_yolo}_normal"

# model = YOLO(weights_path)
# model.train(
#     data=data_yaml_path,
#     epochs=epoch_number,
#     batch=40,
#     device='cuda',
#     project=saved_path,
#     name=results_folder_name,
#     patience=epoch_number,
#     pretrained=True,
#     lr0=0.01,
#     lrf=0.001,
#     dropout=0.2
# )

In [ ]:
data_yaml_path = '/content/drive/My Drive/2024 Fall/COMS 6730/Projects/Senior-Design-VIAD/data.yaml'
with open(data_yaml_path, 'r') as f:
    print(f.read())  

In [ ]:
def pgd_attack(model, images, epsilon, alpha, num_iter, target_class=13):
    images_adv = images.clone().detach().requires_grad_(True)

    with torch.enable_grad():
        for i in range(num_iter):
            images_adv.retain_grad()
            outputs = model(images_adv)

            confidences = outputs[0].boxes.conf 
            # print(confidences)
            if target_class is not None:
                classes = outputs[0].boxes.cls
                target_conf = confidences[classes == target_class]
                if len(target_conf) == 0: 
                    continue
                Closs = -torch.mean(target_conf)  
            else:
                Closs = torch.mean(confidences)  

            # loss = -images_adv.mean()
            loss = images_adv.mean() - Closs
            print(loss)

            loss.requires_grad_(True)
            loss.backward(retain_graph=True)

            if images_adv.grad is None:
                print("images_adv.grad is None after backward")
                break

            grad_sign = images_adv.grad.sign()

            images_adv = images_adv + alpha * grad_sign

            eta = torch.clamp(images_adv - images, min=-epsilon, max=epsilon)
            images_adv = torch.clamp(images + eta, min=0, max=3).detach()

            images_adv.requires_grad_(True)

    return images_adv

In [ ]:
import random

def draw_boxes(image, outputs, class_names=None, color=(0, 255, 0)):

    image_with_boxes = image.copy()

    boxes = outputs[0].boxes

    if boxes is None:
        print("No objects detected.")
        return image_with_boxes

    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())  
        conf = float(box.conf[0]) 
        cls = int(box.cls[0])  

        if int(cls)==13:
          color=(0, 255, 0)
        else:
          color = (int(cls)*10, int(cls)*10, int(cls)*10)

        if class_names and cls < len(class_names):
            label = f"{class_names[cls]}: {conf:.2f}"
        else:
            label = f"Class {cls}: {conf:.2f}" 

        cv2.rectangle(image_with_boxes, (x1, y1), (x2, y2), color, 2)
        print(f"{conf:.2f}")
        label = f"{class_names[cls] if class_names else cls}: {conf:.2f}"
        cv2.putText(image_with_boxes, label, (x1 + 10, y1 + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return image_with_boxes

In [ ]:
def preprocess_image(image, size=(640, 640)):

    image_resized = cv2.resize(image, size)
    image_tensor = torch.tensor(image_resized, dtype=torch.float32).permute(2, 0, 1) / 255.0

    return image_tensor.unsqueeze(0)

In [ ]:
def attack_and_show_image(model, image_path, epsilon, alpha, num_iter, class_range, size=(640, 640)):
  image = cv2.imread(image_path)
  outputs = model(image)
  
  # images = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0).to(device) / 255.0
  # labels = torch.tensor([0]).to(device)
  images = preprocess_image(image, size=(640, 640))

  print("Generating adversarial examples...")
  images_adv = pgd_attack(model, images, epsilon, alpha, num_iter)

  outputs_adv = model(images_adv)

  image_with_boxes = draw_boxes(image, outputs, class_range)

  plt.imshow(cv2.cvtColor(image_with_boxes, cv2.COLOR_BGR2RGB))
  plt.axis("off")
  plt.show()

  adv_img_np = images_adv.squeeze(0).permute(1, 2, 0).detach().cpu().numpy()
  adv_img_np = (adv_img_np * 255).astype('uint8')

  image_with_boxes = draw_boxes(adv_img_np, outputs_adv, class_range)

  plt.imshow(cv2.cvtColor(image_with_boxes, cv2.COLOR_BGR2RGB))
  plt.axis("off")
  plt.show()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = f"{results_folder_name}/weights/last.pt"
model = YOLO(model_path)

with torch.no_grad():
    for param in model.parameters():
        param.requires_grad = False

epsilon = 0.05 
alpha = 0.001  
num_iter = 100  
class_range = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]

In [ ]:
folder_path = '/content/drive/My Drive/2024 Fall/COMS 6730/Projects/Person-Detection-YOLOV8N-Dectection/valid/images'
image_paths = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.endswith('.jpg')]

for image_path in image_paths:
    attack_and_show_image(model, image_path, epsilon, alpha, num_iter, class_range, size=(640, 640))

In [ ]:
folder_path = '/content/drive/My Drive/2024 Fall/COMS 6730/Projects/Person-Detection-YOLOV8N-Dectection/test/images'
image_paths = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.endswith('.jpg')]

for image_path in image_paths:
    attack_and_show_image(model, image_path, epsilon, alpha, num_iter, class_range, size=(640, 640))